In [1]:
import pandas as pd
import numpy as np
import pickle
import csv
import time
import os
from numba import njit
os.chdir('..')
from qmc import QMC
os.chdir('./data')

Load data in the format used by k. Train.

In [2]:
d = pd.read_csv('catsup_trainformat.csv', delimiter=',')
d.head(10)

,chosen,id,alt,display,feature,price,chid
0,0,1,0,0,0,4.6,0
1,0,1,1,0,0,3.7,0
2,1,1,2,0,0,5.2,0
3,0,1,3,0,0,3.4,0
4,0,1,0,0,0,4.6,1
5,0,1,1,0,0,4.3,1
6,1,1,2,0,0,5.2,1
7,0,1,3,0,0,4.4,1
8,0,1,0,0,0,4.6,2
9,0,1,1,0,1,2.5,2


In [3]:
X = d.values[:,1:-1]
X.shape

(11192, 5)

In [4]:
def utilities(X, beta):
    #performs matrix product to obtain the probability of every row
    #X should be in format [display, feature, price]
    try:
        assert(X.shape == (11192, 5) and beta.shape == (3,300))
    except AssertionError:
        raise AssertionError('Ga X ff in juiste format gooien. X: %s, beta: %s' %(X.shape,beta.shape))
    beta_choice = np.zeros((3,11192))
    for i in range(11192):
        id = int(X[i,0])
        beta_choice[:,i] = beta[:,id-1]
    eps = np.random.gumbel(size=(11192,))
    P = (X[:,2:]@ beta_choice)[:,0] + eps
#     try:
#         p = P[0:4]
#         check = p/np.sum(p)
#         assert(np.sum(check) == 1. or np.sum(check) == 1)
#     except AssertionError:
#         raise AssertionError('Kansen van eerste aankoop sommeren niet naar 1 %f'%(np.sum(check)))

    Y = []
    for i in range(0,len(P), 4):
        choice = np.argmax(P[i:i+4])
#         p = P[i:i+4]
#         check = p/np.sum(p)
#         print(np.sum(check))
        Y.append(int(choice))
    return np.array(Y)

In [21]:
# @njit
def probs(X,beta):
    #performs matrix product to obtain the probability of every row
    #X should be in format [display, feature, price]
    if not (X.shape == (11192, 5) and beta.shape == (3,300)):
        raise AssertionError('Ga X ff in juiste format gooien.')
    beta_choice = np.zeros((3,11192))
    for i in range(11192):
        id = int(X[i,0])
        beta_choice[:,i] = beta[:,id-1]   
    P = np.exp((X[:,2:]@ beta_choice)[:,0])
        
    if not (P.shape == (11192,)):
        raise AssertionError('Product van X en beta gaat niet goed.')
#     try:
#         p = P[0:4]
#         check = p/np.sum(p)
#         assert(np.sum(check) == 1. or np.sum(check) == 1)
#     except AssertionError:
#         raise AssertionError('Kansen van eerste aankoop sommeren niet naar 1 %f'%(np.sum(check)))

    Y = np.zeros((2798,))
    for i in range(0,11192, 4):
        sum = P[i:i+4].sum(axis=0)
        Y[i//4] = (P[i:i+4]/sum).argmax(axis=0)
#         p = P[i:i+4]
#         check = p/np.sum(p)
#         print(np.sum(check))
    return Y

In [22]:
%%time
def dgp(X: np.ndarray, D, calcer, dist = np.random.standard_normal):
    #X: dataset
    #D: amount of datasets
    np.random.seed(123)
    theta = np.array([1.5,  1.,  -1.1,  0.4,  0.1,  0.6])
    the_big_dict = {}
    Y_array = np.zeros((2798,D))
    if dist == QMC:
        delta = dist((300,3,D))
        method = 'QMC'
    elif dist == np.random.standard_normal:
        delta = dist((300,3,D))
        method = 'MC'
    print(method)
    for i in range(D):
        delta_d = delta[:,:,i].T
        beta = theta[:K][:, None] + delta_d * theta[K:][:, None]
        Y = calcer(X,beta)
        Y_array[:,i] = Y
    the_big_dict['theta: %s'%(theta)] = Y_array
    return the_big_dict, method
    

Wall time: 0 ns


In [23]:
#display_mean, feature_mean, price_mean, display_sigma, feature_sigma, price_disgma
N = X.shape[0]
K = 3

In [24]:
%%time
D = 500
dicter, method = dgp(X,D,probs)
pickle.dump( dicter, open( "%i_%s_dgp_probs.p"%(D,method), "wb" ) )

MC
Wall time: 6min


In [31]:
dgp_uts = pickle.load(open('500_MC_dgp_uts.p', 'rb'))
dgp_probs = pickle.load(open('500_MC_dgp_probs.p','rb'))

In [34]:
dgp_probs.values()

dict_values([array([[3., 3., 3., ..., 3., 3., 3.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [3., 3., 3., ..., 3., 3., 3.],
       [3., 3., 3., ..., 3., 3., 3.],
       [2., 2., 2., ..., 2., 2., 2.]])])

In [33]:
np.sum(dgp_probs['theta: [ 1.5  1.  -1.1  0.4  0.1  0.6]'] - dgp_uts['theta: [ 1.5  1.  -1.1  0.4  0.1  0.6]'])

72014.0

In [36]:
dgp_uts.values()

dict_values([array([[3., 3., 3., ..., 3., 1., 1.],
       [0., 3., 0., ..., 0., 3., 2.],
       [1., 1., 1., ..., 1., 1., 2.],
       ...,
       [3., 1., 3., ..., 3., 3., 2.],
       [3., 1., 0., ..., 3., 3., 0.],
       [1., 1., 1., ..., 3., 2., 0.]])])